In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:

%pwd

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config


In [8]:
import os
from textSummarizer.logging import logger

In [9]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_files_exist(self)-> bool:
        try:
            # default status as True
            validation_status = True

            # Obtain all file under directory of artifacts/data_ingestion 
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion"))
            print(f'allll files: {all_files}')
            
            # check files existed or not in ALL_REQUIRED_FILES
            for required_file in self.config.ALL_REQUIRED_FILES:
                print(required_file)
                if required_file not in all_files:
                    validation_status = False
                    break

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation failed due to error: {str(e)}")
            return False
    

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2025-05-01 16:42:25,014: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-01 16:42:25,018: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-01 16:42:25,021: INFO: common: created directory at: artifacts]
[2025-05-01 16:42:25,024: INFO: common: created directory at: artifacts/data_validation]
allll files: ['data.7z', 'licence.txt', 'README.txt', 'test.json', 'train.json', 'val.json']
train.json
test.json
val.json
